In [416]:
import pandas as pd

In [417]:
DATASET_LOCATION="./dataset/expandedDataSet.csv" 

In [418]:
ds = pd.read_csv(DATASET_LOCATION)

In [419]:
ds.head(10)

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.00000000,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.00000000,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.00000000,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.00000000,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.00000000,school_bus,5 - 10,76,78,75
5,5,female,group B,associate's degree,standard,none,married,regularly,yes,1.00000000,school_bus,5 - 10,73,84,79
6,6,female,group B,some college,standard,completed,widowed,never,no,1.00000000,private,5 - 10,85,93,89
7,7,male,group B,some college,free/reduced,none,married,sometimes,yes,1.00000000,private,> 10,41,43,39
8,8,male,group D,high school,free/reduced,completed,single,sometimes,no,3.00000000,private,> 10,65,64,68
9,9,female,group B,high school,free/reduced,none,married,regularly,yes,NaN,private,< 5,37,59,50


<h1><b>DESCRIZIONE DEGLI ATTRIBUTI</b></h1>
<ul>
    <li><b>Gender:</b> Gender of the student (male/female)</li>
    <li><b>EthnicGroup:</b> Ethnic group of the student (group A to E)</li>
    <li><b>ParentEduc:</b> Parent(s) education background (from some_highschool to master's degree)</li>
    <li><b>LunchType:</b> School lunch type (standard or free/reduced)</li>
    <li><b>TestPrep:</b> Test preparation course followed (completed or none)</li>
    <li><b>ParentMaritalStatus:</b> Parent(s) marital status (married/single/widowed/divorced)</li>
    <li><b>PracticeSport:</b> How often the student parctice sport (never/sometimes/regularly))</li>
    <li><b>IsFirstChild:</b> If the child is first child in the family or not (yes/no)</li>
    <li><b>NrSiblings:</b> Number of siblings the student has (0 to 7)</li>
    <li><b>TransportMeans:</b> Means of transport to school (schoolbus/private)</li>
    <li><b>WklyStudyHours:</b> Weekly self-study hours(less that 5hrs; between 5 and 10hrs; more than 10hrs)</li>
    <li><b>MathScore:</b> math test score(0-100)</li>
    <li><b>ReadingScore:</b> reading test score(0-100)</li>
    <li><b>WritingScore:</b> writing test score(0-100)</li>
</ul>


In [420]:
#elimino la colonna TestPrep perchè ha troppi dati mancanti
ds.drop("TestPrep",axis=1,inplace=True)

In [421]:
#elimino la colonna indice perchè non è significativa
ds.drop("Unnamed: 0",axis=1,inplace=True)

In [422]:
#elimino la colonna LunchType perchè non sono sicuro che sia significativa
ds.drop("LunchType",axis=1,inplace=True)

In [423]:
ds.head(10)

,Gender,EthnicGroup,ParentEduc,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,NaN,bachelor's degree,married,regularly,yes,3.00000000,school_bus,< 5,71,71,74
1,female,group C,some college,married,sometimes,yes,0.00000000,NaN,5 - 10,69,90,88
2,female,group B,master's degree,single,sometimes,yes,4.00000000,school_bus,< 5,87,93,91
3,male,group A,associate's degree,married,never,no,1.00000000,NaN,5 - 10,45,56,42
4,male,group C,some college,married,sometimes,yes,0.00000000,school_bus,5 - 10,76,78,75
5,female,group B,associate's degree,married,regularly,yes,1.00000000,school_bus,5 - 10,73,84,79
6,female,group B,some college,widowed,never,no,1.00000000,private,5 - 10,85,93,89
7,male,group B,some college,married,sometimes,yes,1.00000000,private,> 10,41,43,39
8,male,group D,high school,single,sometimes,no,3.00000000,private,> 10,65,64,68
9,female,group B,high school,married,regularly,yes,NaN,private,< 5,37,59,50


In [424]:
ds.shape

(30641, 12)

In [425]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               30641 non-null  object 
 1   EthnicGroup          28801 non-null  object 
 2   ParentEduc           28796 non-null  object 
 3   ParentMaritalStatus  29451 non-null  object 
 4   PracticeSport        30010 non-null  object 
 5   IsFirstChild         29737 non-null  object 
 6   NrSiblings           29069 non-null  float64
 7   TransportMeans       27507 non-null  object 
 8   WklyStudyHours       29686 non-null  object 
 9   MathScore            30641 non-null  int64  
 10  ReadingScore         30641 non-null  int64  
 11  WritingScore         30641 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.8+ MB


In [426]:
#eliminiamo le osservazioni che presentano dati mancanti
ds.dropna(inplace=True)

In [427]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20445 entries, 2 to 30640
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               20445 non-null  object 
 1   EthnicGroup          20445 non-null  object 
 2   ParentEduc           20445 non-null  object 
 3   ParentMaritalStatus  20445 non-null  object 
 4   PracticeSport        20445 non-null  object 
 5   IsFirstChild         20445 non-null  object 
 6   NrSiblings           20445 non-null  float64
 7   TransportMeans       20445 non-null  object 
 8   WklyStudyHours       20445 non-null  object 
 9   MathScore            20445 non-null  int64  
 10  ReadingScore         20445 non-null  int64  
 11  WritingScore         20445 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.0+ MB


In [428]:
#verifichiamo se ci sono delle righe duplicate
ds.drop_duplicates(inplace=True)

In [429]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20445 entries, 2 to 30640
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               20445 non-null  object 
 1   EthnicGroup          20445 non-null  object 
 2   ParentEduc           20445 non-null  object 
 3   ParentMaritalStatus  20445 non-null  object 
 4   PracticeSport        20445 non-null  object 
 5   IsFirstChild         20445 non-null  object 
 6   NrSiblings           20445 non-null  float64
 7   TransportMeans       20445 non-null  object 
 8   WklyStudyHours       20445 non-null  object 
 9   MathScore            20445 non-null  int64  
 10  ReadingScore         20445 non-null  int64  
 11  WritingScore         20445 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.0+ MB


In [430]:
#Mappiamo Gender su un vettore di variabili dummy
ds.Gender.replace({'male':1,'female':0},inplace=True)

In [431]:
ds.head(10)

,Gender,EthnicGroup,ParentEduc,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
2,0,group B,master's degree,single,sometimes,yes,4.00000000,school_bus,< 5,87,93,91
4,1,group C,some college,married,sometimes,yes,0.00000000,school_bus,5 - 10,76,78,75
5,0,group B,associate's degree,married,regularly,yes,1.00000000,school_bus,5 - 10,73,84,79
6,0,group B,some college,widowed,never,no,1.00000000,private,5 - 10,85,93,89
7,1,group B,some college,married,sometimes,yes,1.00000000,private,> 10,41,43,39
8,1,group D,high school,single,sometimes,no,3.00000000,private,> 10,65,64,68
11,1,group D,associate's degree,divorced,sometimes,yes,1.00000000,school_bus,5 - 10,40,52,43
12,0,group B,high school,married,regularly,no,1.00000000,private,5 - 10,66,82,74
13,1,group A,some college,single,sometimes,yes,1.00000000,private,> 10,80,73,71
14,0,group A,master's degree,divorced,sometimes,yes,2.00000000,private,< 5,48,53,58


In [432]:
ds_= pd.get_dummies(ds['EthnicGroup'], prefix='EG')
ds = ds.join(ds_).drop('EthnicGroup',axis=1)

In [433]:
ds.head(10)

,Gender,ParentEduc,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,EG_group A,EG_group B,EG_group C,EG_group D,EG_group E
2,0,master's degree,single,sometimes,yes,4.00000000,school_bus,< 5,87,93,91,False,True,False,False,False
4,1,some college,married,sometimes,yes,0.00000000,school_bus,5 - 10,76,78,75,False,False,True,False,False
5,0,associate's degree,married,regularly,yes,1.00000000,school_bus,5 - 10,73,84,79,False,True,False,False,False
6,0,some college,widowed,never,no,1.00000000,private,5 - 10,85,93,89,False,True,False,False,False
7,1,some college,married,sometimes,yes,1.00000000,private,> 10,41,43,39,False,True,False,False,False
8,1,high school,single,sometimes,no,3.00000000,private,> 10,65,64,68,False,False,False,True,False
11,1,associate's degree,divorced,sometimes,yes,1.00000000,school_bus,5 - 10,40,52,43,False,False,False,True,False
12,0,high school,married,regularly,no,1.00000000,private,5 - 10,66,82,74,False,True,False,False,False
13,1,some college,single,sometimes,yes,1.00000000,private,> 10,80,73,71,True,False,False,False,False
14,0,master's degree,divorced,sometimes,yes,2.00000000,private,< 5,48,53,58,True,False,False,False,False


In [434]:
ds_= pd.get_dummies(ds['ParentEduc'], prefix='ParEduc')
ds = ds.join(ds_).drop('ParentEduc',axis=1)

In [435]:
ds.head(10)

,Gender,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,...,EG_group B,EG_group C,EG_group D,EG_group E,ParEduc_associate's degree,ParEduc_bachelor's degree,ParEduc_high school,ParEduc_master's degree,ParEduc_some college,ParEduc_some high school
2,0,single,sometimes,yes,4.00000000,school_bus,< 5,87,93,91,...,True,False,False,False,False,False,False,True,False,False
4,1,married,sometimes,yes,0.00000000,school_bus,5 - 10,76,78,75,...,False,True,False,False,False,False,False,False,True,False
5,0,married,regularly,yes,1.00000000,school_bus,5 - 10,73,84,79,...,True,False,False,False,True,False,False,False,False,False
6,0,widowed,never,no,1.00000000,private,5 - 10,85,93,89,...,True,False,False,False,False,False,False,False,True,False
7,1,married,sometimes,yes,1.00000000,private,> 10,41,43,39,...,True,False,False,False,False,False,False,False,True,False
8,1,single,sometimes,no,3.00000000,private,> 10,65,64,68,...,False,False,True,False,False,False,True,False,False,False
11,1,divorced,sometimes,yes,1.00000000,school_bus,5 - 10,40,52,43,...,False,False,True,False,True,False,False,False,False,False
12,0,married,regularly,no,1.00000000,private,5 - 10,66,82,74,...,True,False,False,False,False,False,True,False,False,False
13,1,single,sometimes,yes,1.00000000,private,> 10,80,73,71,...,False,False,False,False,False,False,False,False,True,False
14,0,divorced,sometimes,yes,2.00000000,private,< 5,48,53,58,...,False,False,False,False,False,False,False,True,False,False


In [436]:
ds_= pd.get_dummies(ds['ParentMaritalStatus'], prefix='ParMarStat')
ds = ds.join(ds_).drop('ParentMaritalStatus',axis=1)

In [437]:
ds.head(10)

,Gender,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,EG_group A,...,ParEduc_associate's degree,ParEduc_bachelor's degree,ParEduc_high school,ParEduc_master's degree,ParEduc_some college,ParEduc_some high school,ParMarStat_divorced,ParMarStat_married,ParMarStat_single,ParMarStat_widowed
2,0,sometimes,yes,4.00000000,school_bus,< 5,87,93,91,False,...,False,False,False,True,False,False,False,False,True,False
4,1,sometimes,yes,0.00000000,school_bus,5 - 10,76,78,75,False,...,False,False,False,False,True,False,False,True,False,False
5,0,regularly,yes,1.00000000,school_bus,5 - 10,73,84,79,False,...,True,False,False,False,False,False,False,True,False,False
6,0,never,no,1.00000000,private,5 - 10,85,93,89,False,...,False,False,False,False,True,False,False,False,False,True
7,1,sometimes,yes,1.00000000,private,> 10,41,43,39,False,...,False,False,False,False,True,False,False,True,False,False
8,1,sometimes,no,3.00000000,private,> 10,65,64,68,False,...,False,False,True,False,False,False,False,False,True,False
11,1,sometimes,yes,1.00000000,school_bus,5 - 10,40,52,43,False,...,True,False,False,False,False,False,True,False,False,False
12,0,regularly,no,1.00000000,private,5 - 10,66,82,74,False,...,False,False,True,False,False,False,False,True,False,False
13,1,sometimes,yes,1.00000000,private,> 10,80,73,71,True,...,False,False,False,False,True,False,False,False,True,False
14,0,sometimes,yes,2.00000000,private,< 5,48,53,58,True,...,False,False,False,True,False,False,True,False,False,False


In [438]:
ds_= pd.get_dummies(ds['PracticeSport'], prefix='sport')
ds = ds.join(ds_).drop('PracticeSport',axis=1)

In [439]:
ds.head(10)

,Gender,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,EG_group A,EG_group B,...,ParEduc_master's degree,ParEduc_some college,ParEduc_some high school,ParMarStat_divorced,ParMarStat_married,ParMarStat_single,ParMarStat_widowed,sport_never,sport_regularly,sport_sometimes
2,0,yes,4.00000000,school_bus,< 5,87,93,91,False,True,...,True,False,False,False,False,True,False,False,False,True
4,1,yes,0.00000000,school_bus,5 - 10,76,78,75,False,False,...,False,True,False,False,True,False,False,False,False,True
5,0,yes,1.00000000,school_bus,5 - 10,73,84,79,False,True,...,False,False,False,False,True,False,False,False,True,False
6,0,no,1.00000000,private,5 - 10,85,93,89,False,True,...,False,True,False,False,False,False,True,True,False,False
7,1,yes,1.00000000,private,> 10,41,43,39,False,True,...,False,True,False,False,True,False,False,False,False,True
8,1,no,3.00000000,private,> 10,65,64,68,False,False,...,False,False,False,False,False,True,False,False,False,True
11,1,yes,1.00000000,school_bus,5 - 10,40,52,43,False,False,...,False,False,False,True,False,False,False,False,False,True
12,0,no,1.00000000,private,5 - 10,66,82,74,False,True,...,False,False,False,False,True,False,False,False,True,False
13,1,yes,1.00000000,private,> 10,80,73,71,True,False,...,False,True,False,False,False,True,False,False,False,True
14,0,yes,2.00000000,private,< 5,48,53,58,True,False,...,True,False,False,True,False,False,False,False,False,True


In [440]:
ds.IsFirstChild.replace({'yes':1,'no':0},inplace=True)

In [441]:
ds.TransportMeans.replace({'school_bus':1,'private':0},inplace=True)

In [442]:
ds_= pd.get_dummies(ds['WklyStudyHours'], prefix='studyHours')
ds = ds.join(ds_).drop('WklyStudyHours',axis=1)

In [443]:
ds.head(10)

,Gender,IsFirstChild,NrSiblings,TransportMeans,MathScore,ReadingScore,WritingScore,EG_group A,EG_group B,EG_group C,...,ParMarStat_divorced,ParMarStat_married,ParMarStat_single,ParMarStat_widowed,sport_never,sport_regularly,sport_sometimes,studyHours_5 - 10,studyHours_< 5,studyHours_> 10
2,0,1,4.00000000,1,87,93,91,False,True,False,...,False,False,True,False,False,False,True,False,True,False
4,1,1,0.00000000,1,76,78,75,False,False,True,...,False,True,False,False,False,False,True,True,False,False
5,0,1,1.00000000,1,73,84,79,False,True,False,...,False,True,False,False,False,True,False,True,False,False
6,0,0,1.00000000,0,85,93,89,False,True,False,...,False,False,False,True,True,False,False,True,False,False
7,1,1,1.00000000,0,41,43,39,False,True,False,...,False,True,False,False,False,False,True,False,False,True
8,1,0,3.00000000,0,65,64,68,False,False,False,...,False,False,True,False,False,False,True,False,False,True
11,1,1,1.00000000,1,40,52,43,False,False,False,...,True,False,False,False,False,False,True,True,False,False
12,0,0,1.00000000,0,66,82,74,False,True,False,...,False,True,False,False,False,True,False,True,False,False
13,1,1,1.00000000,0,80,73,71,True,False,False,...,False,False,True,False,False,False,True,False,False,True
14,0,1,2.00000000,0,48,53,58,True,False,False,...,True,False,False,False,False,False,True,False,True,False


In [444]:
#Analisi della correlazione (Da fare)
ds.corr()

,Gender,IsFirstChild,NrSiblings,TransportMeans,MathScore,ReadingScore,WritingScore,EG_group A,EG_group B,EG_group C,...,ParMarStat_divorced,ParMarStat_married,ParMarStat_single,ParMarStat_widowed,sport_never,sport_regularly,sport_sometimes,studyHours_5 - 10,studyHours_< 5,studyHours_> 10
Gender,1.00000000,0.00753004,0.00001940,0.00927157,0.15818373,-0.24117547,-0.29184370,0.00036623,0.00177390,-0.00482245,...,-0.01787971,0.02386235,-0.01206648,0.00030784,0.00407022,-0.01101161,0.00780583,-0.00216240,-0.00471758,0.00842791
IsFirstChild,0.00753004,1.00000000,-0.11851838,-0.00617200,0.01085489,0.00778622,0.00292951,-0.00610826,0.00961967,-0.00160848,...,0.00631496,0.00413861,-0.01079921,0.00141883,-0.00747458,0.01265724,-0.00706805,0.00364653,-0.00260665,-0.00170216
NrSiblings,0.00001940,-0.11851838,1.00000000,-0.00700297,0.00141171,0.00002279,0.00049679,-0.00322471,-0.00327025,0.00960277,...,0.00859453,-0.00254124,-0.00576010,0.00358478,0.01015720,-0.01493737,0.00743134,-0.00176164,-0.00147710,0.00406337
TransportMeans,0.00927157,-0.00617200,-0.00700297,1.00000000,0.00646781,0.00120737,0.00111129,-0.01536109,0.00918798,-0.01234796,...,-0.00156506,-0.00147709,0.00239442,0.00213028,-0.00209993,-0.00234061,0.00367862,-0.00456585,0.01223846,-0.00849865
MathScore,0.15818373,0.01085489,0.00141171,0.00646781,1.00000000,0.81935856,0.80876268,-0.06713600,-0.10125040,-0.08788177,...,0.00249800,0.00719171,-0.01347129,0.00907285,-0.05500143,0.05221841,-0.01270222,0.02401421,-0.08185065,0.06540536
ReadingScore,-0.24117547,0.00778622,0.00002279,0.00120737,0.81935856,1.00000000,0.95252939,-0.05451955,-0.07427119,-0.04513283,...,0.00705391,-0.00124248,-0.00695709,0.00680189,-0.02353147,0.02392329,-0.00695459,0.01957055,-0.04715418,0.03014610
WritingScore,-0.29184370,0.00292951,0.00049679,0.00111129,0.80876268,0.95252939,1.00000000,-0.06423266,-0.08491209,-0.06447669,...,0.00932440,-0.00466159,-0.00459731,0.00560158,-0.04397113,0.05367244,-0.02161133,0.01422949,-0.05015291,0.04071318
EG_group A,0.00036623,-0.00610826,-0.00322471,-0.01536109,-0.06713600,-0.05451955,-0.06423266,1.00000000,-0.14508163,-0.19761705,...,0.00427809,-0.00204029,-0.00289997,0.00465546,-0.01120616,-0.00412489,0.01159454,0.00316674,-0.00467872,0.00138228
EG_group B,0.00177390,0.00961967,-0.00327025,0.00918798,-0.10125040,-0.07427119,-0.08491209,-0.14508163,1.00000000,-0.34732075,...,-0.01089467,-0.00181027,0.01344522,-0.00550469,0.00603257,-0.00720405,0.00281175,0.00129416,-0.00048599,-0.00112419
EG_group C,-0.00482245,-0.00160848,0.00960277,-0.01234796,-0.08788177,-0.04513283,-0.06447669,-0.19761705,-0.34732075,1.00000000,...,-0.00585646,0.00897304,-0.00490197,-0.00110558,-0.00246088,0.00259139,-0.00081331,0.00525824,0.00584104,-0.01382483


In [445]:
ds_copy = ds.copy() # questa riga serve per avere il dataset per provare la standardizzazione di sklearn ( alla fine)

In [446]:
#Reading e writing sono altamente correlati, li aggreghiamo facendone la media
new = pd.Series((ds.ReadingScore+ds.WritingScore)/2,name='RWScore')

In [447]:
ds.drop(['ReadingScore','WritingScore'],axis=1,inplace=True)
ds = ds.join(new)

In [448]:
ds.head(10)

,Gender,IsFirstChild,NrSiblings,TransportMeans,MathScore,EG_group A,EG_group B,EG_group C,EG_group D,EG_group E,...,ParMarStat_married,ParMarStat_single,ParMarStat_widowed,sport_never,sport_regularly,sport_sometimes,studyHours_5 - 10,studyHours_< 5,studyHours_> 10,RWScore
2,0,1,4.00000000,1,87,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,92.00000000
4,1,1,0.00000000,1,76,False,False,True,False,False,...,True,False,False,False,False,True,True,False,False,76.50000000
5,0,1,1.00000000,1,73,False,True,False,False,False,...,True,False,False,False,True,False,True,False,False,81.50000000
6,0,0,1.00000000,0,85,False,True,False,False,False,...,False,False,True,True,False,False,True,False,False,91.00000000
7,1,1,1.00000000,0,41,False,True,False,False,False,...,True,False,False,False,False,True,False,False,True,41.00000000
8,1,0,3.00000000,0,65,False,False,False,True,False,...,False,True,False,False,False,True,False,False,True,66.00000000
11,1,1,1.00000000,1,40,False,False,False,True,False,...,False,False,False,False,False,True,True,False,False,47.50000000
12,0,0,1.00000000,0,66,False,True,False,False,False,...,True,False,False,False,True,False,True,False,False,78.00000000
13,1,1,1.00000000,0,80,True,False,False,False,False,...,False,True,False,False,False,True,False,False,True,72.00000000
14,0,1,2.00000000,0,48,True,False,False,False,False,...,False,False,False,False,False,True,False,True,False,55.50000000


In [449]:
ds.corr()

,Gender,IsFirstChild,NrSiblings,TransportMeans,MathScore,EG_group A,EG_group B,EG_group C,EG_group D,EG_group E,...,ParMarStat_married,ParMarStat_single,ParMarStat_widowed,sport_never,sport_regularly,sport_sometimes,studyHours_5 - 10,studyHours_< 5,studyHours_> 10,RWScore
Gender,1.00000000,0.00753004,0.00001940,0.00927157,0.15818373,0.00036623,0.00177390,-0.00482245,0.00496008,-0.00210254,...,0.02386235,-0.01206648,0.00030784,0.00407022,-0.01101161,0.00780583,-0.00216240,-0.00471758,0.00842791,-0.27032821
IsFirstChild,0.00753004,1.00000000,-0.11851838,-0.00617200,0.01085489,-0.00610826,0.00961967,-0.00160848,-0.00345239,0.00003834,...,0.00413861,-0.01079921,0.00141883,-0.00747458,0.01265724,-0.00706805,0.00364653,-0.00260665,-0.00170216,0.00536504
NrSiblings,0.00001940,-0.11851838,1.00000000,-0.00700297,0.00141171,-0.00322471,-0.00327025,0.00960277,-0.00866609,0.00426346,...,-0.00254124,-0.00576010,0.00358478,0.01015720,-0.01493737,0.00743134,-0.00176164,-0.00147710,0.00406337,0.00026854
TransportMeans,0.00927157,-0.00617200,-0.00700297,1.00000000,0.00646781,-0.01536109,0.00918798,-0.01234796,0.00905256,0.00627244,...,-0.00147709,0.00239442,0.00213028,-0.00209993,-0.00234061,0.00367862,-0.00456585,0.01223846,-0.00849865,0.00117219
MathScore,0.15818373,0.01085489,0.00141171,0.00646781,1.00000000,-0.06713600,-0.10125040,-0.08788177,0.04507744,0.22960499,...,0.00719171,-0.01347129,0.00907285,-0.05500143,0.05221841,-0.01270222,0.02401421,-0.08185065,0.06540536,0.82376607
EG_group A,0.00036623,-0.00610826,-0.00322471,-0.01536109,-0.06713600,1.00000000,-0.14508163,-0.19761705,-0.16970403,-0.11629195,...,-0.00204029,-0.00289997,0.00465546,-0.01120616,-0.00412489,0.01159454,0.00316674,-0.00467872,0.00138228,-0.06020820
EG_group B,0.00177390,0.00961967,-0.00327025,0.00918798,-0.10125040,-0.14508163,1.00000000,-0.34732075,-0.29826238,-0.20438827,...,-0.00181027,0.01344522,-0.00550469,0.00603257,-0.00720405,0.00281175,0.00129416,-0.00048599,-0.00112419,-0.08067886
EG_group C,-0.00482245,-0.00160848,0.00960277,-0.01234796,-0.08788177,-0.19761705,-0.34732075,1.00000000,-0.40626597,-0.27839918,...,0.00897304,-0.00490197,-0.00110558,-0.00246088,0.00259139,-0.00081331,0.00525824,0.00584104,-0.01382483,-0.05569572
EG_group D,0.00496008,-0.00345239,-0.00866609,0.00905256,0.04507744,-0.16970403,-0.29826238,-0.40626597,1.00000000,-0.23907584,...,-0.00537718,-0.00722657,-0.00440190,0.00396530,0.00330221,-0.00587277,-0.01346840,0.00193408,0.01539939,0.07034502
EG_group E,-0.00210254,0.00003834,0.00426346,0.00627244,0.22960499,-0.11629195,-0.20438827,-0.27839918,-0.23907584,1.00000000,...,-0.00162511,0.00233782,0.00984262,-0.00011638,0.00384563,-0.00361490,0.00598455,-0.00614442,-0.00058278,0.12549191


In [450]:
#Dividiamo i valori delle valutazioni per 10
#portandole così da un intervallo 0-100 a un intervallo 0-10
mat2 =pd.Series(ds.MathScore/10,name='MathScore')
ds.drop('MathScore',axis=1,inplace=True)
ds = ds.join(mat2)
rw2 =pd.Series(ds.RWScore/10,name='RWScore')
ds.drop('RWScore',axis=1,inplace=True)
ds = ds.join(rw2)

In [451]:
import numpy as np
np.random.seed(50)

In [452]:
#Definiamo la funzione compute_performance così da poter poi stampare
#la tabella con le varie perfomance dei diversi modelli
#from sklearn.model_selection import cross_val_predict, cross_val_score

import time

pd.options.display.float_format = '{:.8f}'.format

def compute_performance(modelli,names,X,y):
    score_dict = {}
    score_dict['Modello'] = {}
    score_dict['mse'] = {}
    score_dict['variance'] = {}
    score_dict['mae'] = {}
    score_dict['median_ae'] = {}
    score_dict['r2'] = {}
    score_dict['mape'] = {}
    

    for i in range(len(modelli)):
        value_predictions = modelli[i].predict(X)
        mse = mean_squared_error(y, value_predictions)
        variance = explained_variance_score(y,value_predictions)
        mae = mean_absolute_error(y,value_predictions)
        mape = mean_absolute_percentage_error(y, value_predictions)
        median_ae = median_absolute_error(y, value_predictions)
        r2 = r2_score(y, value_predictions)

        name = names[i]

        score_dict['Modello'][i] = name
        score_dict['mse'][i] = mse
        score_dict['variance'][i] = variance
        score_dict['mae'][i] = mae
        score_dict['median_ae'][i] = median_ae
        score_dict['r2'][i] = r2
        score_dict['mape'][i] = mape
        
    return score_dict

In [453]:
regressori = []
nomi = []

<h1> Modello SVM </h1>

In [454]:
modelli_svm = []

In [455]:
#suddivisione del dataset in test set e training set
from sklearn.model_selection import train_test_split
from sklearn.metrics import*
X = ds.drop('MathScore',axis=1)
Y = ds['MathScore']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [456]:
from sklearn.svm import SVR
svr_reg = SVR()
svr_reg.fit(X_train,y_train)

SVR()

In [457]:
#performance della SVR sul training set
#y_pred_svr= svr_reg.predict(X_train)
'''mse = mean_squared_error(y_train,y_pred_svr)
var = explained_variance_score(y_train,y_pred_svr)
mae = mean_absolute_error(y_train,y_pred_svr)
median_ae = median_absolute_error(y_train,y_pred_svr)
r2 = r2_score(y_train,y_pred_svr)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_train,y_pred_svr)\nvar = explained_variance_score(y_train,y_pred_svr)\nmae = mean_absolute_error(y_train,y_pred_svr)\nmedian_ae = median_absolute_error(y_train,y_pred_svr)\nr2 = r2_score(y_train,y_pred_svr)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [458]:
#performance della SVR sul test set
#y_pred_svr= svr_reg.predict(X_test)
'''mse = mean_squared_error(y_test,y_pred_svr)
var = explained_variance_score(y_test,y_pred_svr)
mae = mean_absolute_error(y_test,y_pred_svr)
median_ae = median_absolute_error(y_test,y_pred_svr)
r2 = r2_score(y_test,y_pred_svr)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_test,y_pred_svr)\nvar = explained_variance_score(y_test,y_pred_svr)\nmae = mean_absolute_error(y_test,y_pred_svr)\nmedian_ae = median_absolute_error(y_test,y_pred_svr)\nr2 = r2_score(y_test,y_pred_svr)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [459]:
regressori.append(svr_reg)
nomi.append("SVR Regressor")
modelli_svm.append("SVR Regressor")

<h1>Modelli Lineari</h1></n>

In [460]:
modelli_lineari = []

<h2>Linear Regressor</h2>

In [461]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X_train, y_train)

In [462]:
#performance del linear regressor sul training set
#y_pred_lin= lin_reg.predict(X_train)
'''mse = mean_squared_error(y_train,y_pred_lin)
var = explained_variance_score(y_train,y_pred_lin)
mae = mean_absolute_error(y_train,y_pred_lin)
median_ae = median_absolute_error(y_train,y_pred_lin)
r2 = r2_score(y_train,y_pred_lin)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_train,y_pred_lin)\nvar = explained_variance_score(y_train,y_pred_lin)\nmae = mean_absolute_error(y_train,y_pred_lin)\nmedian_ae = median_absolute_error(y_train,y_pred_lin)\nr2 = r2_score(y_train,y_pred_lin)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [463]:
#performance del linear regressor sul test set
#y_pred_lin= lin_reg.predict(X_test)
'''mse = mean_squared_error(y_test,y_pred_lin)
var = explained_variance_score(y_test,y_pred_lin)
mae = mean_absolute_error(y_test,y_pred_lin)
median_ae = median_absolute_error(y_test,y_pred_lin)
r2 = r2_score(y_test,y_pred_lin)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_test,y_pred_lin)\nvar = explained_variance_score(y_test,y_pred_lin)\nmae = mean_absolute_error(y_test,y_pred_lin)\nmedian_ae = median_absolute_error(y_test,y_pred_lin)\nr2 = r2_score(y_test,y_pred_lin)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [464]:
regressori.append(lin_reg)
nomi.append("Linear Regressor")
modelli_lineari.append("Linear Regressor")

<h1>SGD Regressor</h1>

In [465]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor() #maybe random_state=42
sgd_reg.fit(X_train, y_train)

SGDRegressor()

In [466]:
#performance del linear regressor sul training set
#y_pred_sgd= sgd_reg.predict(X_train)
'''mse = mean_squared_error(y_train,y_pred_sgd)
var = explained_variance_score(y_train,y_pred_sgd)
mae = mean_absolute_error(y_train,y_pred_sgd)
median_ae = median_absolute_error(y_train,y_pred_sgd)
r2 = r2_score(y_train,y_pred_sgd)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_train,y_pred_sgd)\nvar = explained_variance_score(y_train,y_pred_sgd)\nmae = mean_absolute_error(y_train,y_pred_sgd)\nmedian_ae = median_absolute_error(y_train,y_pred_sgd)\nr2 = r2_score(y_train,y_pred_sgd)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [467]:
#performance del linear regressor sul test set
#y_pred_sgd= sgd_reg.predict(X_test)
'''mse = mean_squared_error(y_test,y_pred_sgd)
var = explained_variance_score(y_test,y_pred_sgd)
mae = mean_absolute_error(y_test,y_pred_sgd)
median_ae = median_absolute_error(y_test,y_pred_sgd)
r2 = r2_score(y_test,y_pred_sgd)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_test,y_pred_sgd)\nvar = explained_variance_score(y_test,y_pred_sgd)\nmae = mean_absolute_error(y_test,y_pred_sgd)\nmedian_ae = median_absolute_error(y_test,y_pred_sgd)\nr2 = r2_score(y_test,y_pred_sgd)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [468]:
regressori.append(sgd_reg)
nomi.append("SGD Regressor")
modelli_lineari.append("SGD Regressor")

<h1>Ensemble Models</h1>

In [469]:
modelli_ensemble = []

<h2>Random Forest</h2>

In [470]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor() #n_estimators=10, random_state=42
forest_reg.fit(X_train, y_train)

RandomForestRegressor()

In [471]:
#performance del forest regressor sul training set
#y_pred_forest= forest_reg.predict(X_train)
'''mse = mean_squared_error(y_train,y_pred_forest)
var = explained_variance_score(y_train,y_pred_forest)
mae = mean_absolute_error(y_train,y_pred_forest)
median_ae = median_absolute_error(y_train,y_pred_forest)
r2 = r2_score(y_train,y_pred_forest)
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)'''

'mse = mean_squared_error(y_train,y_pred_forest)\nvar = explained_variance_score(y_train,y_pred_forest)\nmae = mean_absolute_error(y_train,y_pred_forest)\nmedian_ae = median_absolute_error(y_train,y_pred_forest)\nr2 = r2_score(y_train,y_pred_forest)\nprint("mse",mse)\nprint("var",var)\nprint("mae",mae)\nprint("median_ae",median_ae)\nprint("r2",r2)'

In [472]:
regressori.append(forest_reg)
nomi.append("Forest Regressor")
modelli_lineari.append("Forest Regressor")

<h2>AdaBoost Regressor</h2>

In [474]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor()
ada_reg.fit(X_train, y_train)

AdaBoostRegressor()

In [475]:
regressori.append(ada_reg)
nomi.append("Ada Regressor")
modelli_ensemble.append("Ada Regressor")

<h2>Neural networks</h2>

In [476]:
reti_neurali = []

In [477]:
from sklearn.neural_network import MLPRegressor

mlp_reg = MLPRegressor()
mlp_reg.fit(X_train, y_train)

MLPRegressor()

In [478]:
regressori.append(mlp_reg)
nomi.append("MLP Regressor")
reti_neurali.append("MLP Regressor")

<h1>Modelli ad Albero</h1>


In [479]:
modelli_alberi = []

<h2>DecisionTree Regressor</h2>

In [480]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [481]:
regressori.append(tree_reg)
nomi.append("DecisionTree Regressor")
modelli_alberi.append("DecisionTree Regressor")

<h1>Regressori basati su Istance</h1>

In [482]:
modelli_istance = []

<h2>K Neighbour Regressor</h2>

In [483]:
from sklearn.neighbors import KNeighborsRegressor

kn_reg = KNeighborsRegressor()
kn_reg.fit(X_train, y_train)

KNeighborsRegressor()

In [484]:
regressori.append(kn_reg)
nomi.append("K Neighbor Regressor")
modelli_istance.append("K Neighbor Regressor")

<h1>Valutazione performance modelli - Training set</h1>

In [485]:
performance_modelli_training = compute_performance(regressori, nomi, X_train,y_train)
performance_train = pd.DataFrame(performance_modelli_training)
performance_train

,Modello,mse,variance,mae,median_ae,r2,mape
0,SVR Regressor,0.34630203,0.85316174,0.46809031,0.39519955,0.85315922,387082399258.85601807
1,Linear Regressor,0.35091016,0.85120527,0.47349986,0.40117187,0.85120525,236762041637.82916260
2,SGD Regressor,0.36598476,0.85093428,0.48377775,0.41073353,0.84481324,225489721575.09793091
3,Forest Regressor,0.05645266,0.97606276,0.18714107,0.15600000,0.97606265,132993312547.10491943
4,Ada Regressor,0.40539688,0.83036016,0.51122058,0.43705144,0.82810151,442196536098.87878418
5,MLP Regressor,0.32864816,0.86068564,0.45917295,0.38801963,0.86064490,228626127712.96179199
6,DecisionTree Regressor,0.00245843,0.99895757,0.00483003,0.00000000,0.99895757,0.00074862
7,K Neighbor Regressor,0.40625754,0.82773672,0.50753118,0.42000000,0.82773657,726919969201.48486328


<h1> Secondo approccio usando la standardizzazione automatica di SKLEARN</h1>

In [516]:
ds_copy.head() #mean=66 #std = 15

,Gender,IsFirstChild,NrSiblings,TransportMeans,MathScore,ReadingScore,WritingScore,EG_group A,EG_group B,EG_group C,...,ParMarStat_divorced,ParMarStat_married,ParMarStat_single,ParMarStat_widowed,sport_never,sport_regularly,sport_sometimes,studyHours_5 - 10,studyHours_< 5,studyHours_> 10
2,0,1,4.00000000,1,87,93,91,False,True,False,...,False,False,True,False,False,False,True,False,True,False
4,1,1,0.00000000,1,76,78,75,False,False,True,...,False,True,False,False,False,False,True,True,False,False
5,0,1,1.00000000,1,73,84,79,False,True,False,...,False,True,False,False,False,True,False,True,False,False
6,0,0,1.00000000,0,85,93,89,False,True,False,...,False,False,False,True,True,False,False,True,False,False
7,1,1,1.00000000,0,41,43,39,False,True,False,...,False,True,False,False,False,False,True,False,False,True


In [534]:
lista = ds_copy.columns.values.tolist()
diz = {}
for i in range(len(lista)):
    diz[i] = lista[i]
scaler = preprocessing.StandardScaler().fit(ds_copy)
X_scaled = scaler.transform(ds_copy)
dsn = pd.DataFrame(X_scaled)
dsn.rename(diz, axis='columns', inplace=True)

In [535]:
#maperror, standard schedule
from sklearn import preprocessing
X2 = dsn.drop('MathScore',axis=1)
Y2 = dsn['MathScore']
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,Y2, test_size=0.2)

In [536]:
mlp_re2 = MLPRegressor()
mlp_re2.fit(X_train2, y_train2)

MLPRegressor()

In [542]:
y_pred = mlp_re2.predict(X_train2)
mse = mean_squared_error(y_train2,y_pred)
var = explained_variance_score(y_train2,y_pred)
mae = mean_absolute_error(y_train2,y_pred)
median_ae = median_absolute_error(y_train2,y_pred)
mape = mean_absolute_percentage_error(y_train2,y_pred)
r2 = r2_score(y_train2,y_pred)
adjusted_r2 = 1 - ( 1-r2 ) * ( len(y_pred) - 1 ) / ( len(y_pred) - X_train2.shape[1] - 1 )
print("mse",mse)
print("var",var)
print("mae",mae)
print("median_ae",median_ae)
print("r2",r2)
print("mape",mape)
print("adjusted_r2",adjusted_r2)

mse 0.12413008355345014
var 0.8762848226770049
mae 0.28172606191798544
median_ae 0.23896059667566827
r2 0.8762583262426251
mape 1.0224278635301818
adjusted_r2 0.8760537068653929


C:\Users\Nicola\PycharmProjects\ArtificialIntelligence\venv\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


ValueError: X has 26 features, but MLPRegressor is expecting 27 features as input.